<h2>Lab 1 - Scraping and NLP Pipeline</h2>

<hr style="border:2px solid blue;">

<h3>Realised by:</h3>
<ul>
    <li><strong style="color:black;">ELMRABET HANAE</strong></li>
</ul>

<h3>Guided by:</h3> 
   <p><strong style="color:blue;">Pr . ELAACHAk LOTF</strong></p>

<p><strong>Objective :</strong> The main purpose behind this lab is to get familiar with Scraping and NLP
 Pipeline.</p>


<p> In the pursuit of gathering culinary data, I employed BeautifulSoup, a Python library renowned for its adeptness in web scraping, to extract pertinent information from the Fatafeat website. With meticulous parsing of the website's HTML structure, I meticulously harvested a wealth of data including recipe titles, ingredients, preparation methods, and chef attributions. By seamlessly traversing through the intricate HTML elements, I methodically accumulated data from various pages, ensuring a comprehensive dataset for subsequent analysis. BeautifulSoup's robust functionality and versatility facilitated a seamless and efficient scraping process, underscoring its indispensability in acquiring valuable culinary insights from the Fatafeat platform.</p>

#       Sommaire

1. [Import Librairies And Data Loading](#data-loading)
2. [NLP Pipeline](#nlp-pipeline)
    1. [Text Cleaning](#text-cleaning)
    2. [Tokenization](#tokenization)
    3. [Stop words](#stop-words)
    4. [Discretization](#discretization)
    5. [Normalization](#nor)
    6. [ Stemming and Lemmatization](#SL)
3. [Parts of Speech (POS)](#part)
4. [Named Entity Recognition(NER)](#ner)
5. [Lab Learning Recap](#recap)

## 1. Import Librairies And Data Loading<a id='data-loading'></a>

In [268]:
import pymongo
import pandas as pd
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.isri import ISRIStemmer
import nltk
import stanza 


In [269]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["scraping"]
collection = db["scraping1"]

In [270]:
cursor = collection.find()

In [271]:
data = list(cursor)

In [272]:
df = pd.DataFrame(data)

In [273]:
client.close()

In [274]:
df

_id                           titre propriété  \
0    6613405966cd235ae4b7116a  كوكيز‭ ‬رقائق‭ ‬الشوكولاتة‭...      سهلة   
1    6613405966cd235ae4b7116b                   كوكيز قوس قزح    متوسطة   
2    6613405966cd235ae4b7116c    كوكيز زبدة الفول السوداني...    متوسطة   
3    6613405966cd235ae4b7116d       كوكيز الشيكولاتة بالنعناع    متوسطة   
4    6613405966cd235ae4b7116e                   كوكيز البيتزا    متوسطة   
..                        ...                             ...       ...   
125  661342d2885a273d9393c2e2                        ليزي كيك      سهلة   
126  661342d2885a273d9393c2e3     فوندو الشوكولاتة مع الفواكه      سهلة   
127  661342d2885a273d9393c2e4          تارت الشوكولاتة بالتوت      سهلة   
128  661342d2885a273d9393c2e5          تيراميسو الشوكولاتة...    متوسطة   
129  661342d2885a273d9393c2e6   كيفية تحضير حليب الأزر الأبيض      سهلة   

                                           ingredients  \
0    \n\n\n\n 1/2 \n\n\nكوب الدقيق\n\n\n\n\n\n\n 1/...   
1    \n\n\n\n 1/2 \n\n\nكوب الدقيق\n\n\n\n\n\n\n 1/...   
2    \n\n\n\n 0 \n\n\nكوب نصف سكر\n\n\n\n\n\n\n 2 \...   
3    \n\n\n\n 0 \n\n\nكوب نصف سكر\n\n\n\n\n\n\n 2 \...   
4    \n\n\n\n 0 \n\n\nكوب سكر أبيض\n\n\n\n\n\n\n 0 ...   
..                                                 ...   
125  \n\n\n\n 2/3 \n\n\nكوب كريمة\n\n\n\n\n\n\n 120...   
126  \n\n\n\n 250 \n\n\nجم فتات كوكيز الشوكولاته\n\...   
127  \n\n\n\n 250 \n\n\nجم فتات كوكيز الشوكولاته\n\...   
128  \n\n\n\n 1 \n\n\nعلبة بسكويت المملح\n\n\n\n\n\...   
129  \n\n\n\n 1 \n\n\nعلبة بسكويت المملح\n\n\n\n\n\...   

                                              recettes  
0    \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...  
1    \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...  
2    \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...  
3    \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...  
4    \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...  
..                                                 ...  
125  \n\n\n1\n\nيكسر البسكويت ويوضع في وعاء كبير.\n...  
126  \n\n\n1\n\nفي وعاء يحوي شوكولاتة، تضاف الكريمة...  
127  \n\n\n1\n\nيضرب الطحين والكاكاو والزبدة وسكر ا...  
128  \n\n\n1\n\nتخفق جبنة الماسكربون حتى تصبح ناعمة...  
129  \n\n\n1\n\nيغسل الأزر جيداً ويوضع في وعاء. يغم...  

[130 rows x 5 columns]

## 2. NLP Pipeline <a id='nlp-pipeline'></a>

### 2.1 Text Cleaning <a id='text-cleaning'></a>

In [275]:
def clean_arabic_text(text):
    cleaned_text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()
df['titre'] = df['titre'].apply(clean_arabic_text)
df['propriété'] = df['propriété'].apply(clean_arabic_text)
df['ingredients'] = df['ingredients'].apply(clean_arabic_text)
df['recettes'] = df['recettes'].apply(clean_arabic_text)

### 2.3 Stop words <a id='stop-words'></a>

In [276]:
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
df = pd.DataFrame(data)
def remove_stopwords(text):
    tokenizer = WordPunctTokenizer()
    word_tokens = tokenizer.tokenize(text)
    filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_sentence)
df['recipes_without_stopwords'] = df['recettes'].apply(remove_stopwords)
print(df[['recettes', 'recipes_without_stopwords']])

                                              recettes  \
0    \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1    \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2    \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3    \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4    \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   
..                                                 ...   
125  \n\n\n1\n\nيكسر البسكويت ويوضع في وعاء كبير.\n...   
126  \n\n\n1\n\nفي وعاء يحوي شوكولاتة، تضاف الكريمة...   
127  \n\n\n1\n\nيضرب الطحين والكاكاو والزبدة وسكر ا...   
128  \n\n\n1\n\nتخفق جبنة الماسكربون حتى تصبح ناعمة...   
129  \n\n\n1\n\nيغسل الأزر جيداً ويوضع في وعاء. يغم...   

                             recipes_without_stopwords  
0    1 ُ خلط ‭ ‬ نوعي ‭ ‬ الدقيق ‭ ‬ والبيكنج ‭ ‬ ص...  
1    1 يخفق السكر البودرة السكر الأبيض الملح فى وعا...  
2    1 وعاء النار يخلط السكر الشربات الخفيف زبدة ال...  
3    1 لعمل الكوكيز : تخلط المكونات الجافة جيدا ً ....  
4    1 لعمل الصوص 

[nltk_data] Downloading package stopwords to C:\Users\pc
[nltk_data]     hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.4 Discretization <a id='discretization'></a>

In [277]:
bins = [0, 10, 20, 30]  
labels = ['court', 'moyen', 'long']
df['titre_length'] = df['titre'].apply(len) 
df['titre_length_category'] = pd.cut(df['titre_length'], bins=bins, labels=labels)
print(df[['titre', 'titre_length', 'titre_length_category']].head(10))

                            titre  titre_length titre_length_category
0  كوكيز‭ ‬رقائق‭ ‬الشوكولاتة‭...            30                  long
1                   كوكيز قوس قزح            13                 moyen
2    كوكيز زبدة الفول السوداني...            28                  long
3       كوكيز الشيكولاتة بالنعناع            25                  long
4                   كوكيز البيتزا            13                 moyen
5           كوكيز بالزبدة والبندق            21                  long
6     سكونز بالجوز وتوت العليق...            27                  long
7                       كوكيز صحي             9                 court
8              كوكيز بقطرات التمر            18                 moyen
9                     كوكيز التمر            11                 moyen


### 2.5 Normalization <a id='nor'></a>

In [278]:
stemmer = ISRIStemmer()
def stem_arabic_words(word_list):
    return [stemmer.stem(word) for word in word_list]
df['recettes_stemmed'] = df['recettes'].apply(lambda x: stem_arabic_words(x.split()))
print(df[['recettes', 'recettes_stemmed']].head())

                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                    recettes_stemmed  
0  [1, خلط‭, ‬نوعي‭, ‬الدقيق‭, ‬والبيكنج‭, ‬صودا‭...  
1  [1, خفق, سكر, ودر, مع, سكر, ابض, و, ملح, فى, و...  
2  [1, في, وعء, على, نار, خلط, سكر, مع, شرب, خفف,...  
3  [1, عمل, كوكيز, :, خلط, كل, كون, جفة, جيد, ., ...  
4  [1, عمل, صوص, :, في, وعء, توسط, عمق, على, نار,...  


In [279]:
def normalize_arabic_text(text):
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'أ|إ|آ', 'ا', text)
    return text
df['recettes_norm'] = df['recettes'].apply(normalize_arabic_text)
print(df[['recettes', 'recettes_norm']].head())

                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                       recettes_norm  
0  \n\n\n1\n\nيخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬ص...  
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الابيض ...  
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...  
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...  
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...  


### 2.6  Stemming and Lemmatization <a id='SL'></a>

In [280]:
nltk.download('omw-1.4')
stemmer = ISRIStemmer()
def lemmatize_arabic_text(text):
    words = text.split()  
    lemmatized_words = [stemmer.stem(word) for word in words]
    return ' '.join(lemmatized_words)
df['recettes_lemmatized'] = df['recettes'].apply(lemmatize_arabic_text)
print(df[['recettes', 'recettes_lemmatized']].head())

                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                 recettes_lemmatized  
0  1 خلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬صودا‭ ‬والم...  
1  1 خفق سكر ودر مع سكر ابض و ملح فى وعء توسط ، 2...  
2  1 في وعء على نار خلط سكر مع شرب خفف و زبد فول ...  
3  1 عمل كوكيز : خلط كل كون جفة جيد . 2 خفق زبد م...  
4  1 عمل صوص : في وعء توسط عمق على نار خلط سكر مع...  


[nltk_data] Downloading package omw-1.4 to C:\Users\pc
[nltk_data]     hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [281]:
df[['recettes', 'recettes_stemmed', 'recettes_lemmatized']].head()

recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                    recettes_stemmed  \
0  [1, خلط‭, ‬نوعي‭, ‬الدقيق‭, ‬والبيكنج‭, ‬صودا‭...   
1  [1, خفق, سكر, ودر, مع, سكر, ابض, و, ملح, فى, و...   
2  [1, في, وعء, على, نار, خلط, سكر, مع, شرب, خفف,...   
3  [1, عمل, كوكيز, :, خلط, كل, كون, جفة, جيد, ., ...   
4  [1, عمل, صوص, :, في, وعء, توسط, عمق, على, نار,...   

                                 recettes_lemmatized  
0  1 خلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬صودا‭ ‬والم...  
1  1 خفق سكر ودر مع سكر ابض و ملح فى وعء توسط ، 2...  
2  1 في وعء على نار خلط سكر مع شرب خفف و زبد فول ...  
3  1 عمل كوكيز : خلط كل كون جفة جيد . 2 خفق زبد م...  
4  1 عمل صوص : في وعء توسط عمق على نار خلط سكر مع...

### 2.2 Tokenization <a id='tokenization'></a>

In [282]:
tokenizer = WordPunctTokenizer()

df['recettes_tokenized'] = df['recettes'].apply(tokenizer.tokenize)

print(df[['recettes', 'recettes_tokenized']].head())

                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                  recettes_tokenized  
0  [1, ي, ُ, خلط, ‭, ‬, نوعي, ‭, ‬, الدقيق, ‭, ‬,...  
1  [1, يخفق, السكر, البودرة, مع, السكر, الأبيض, و...  
2  [1, في, وعاء, على, النار, يخلط, السكر, مع, الش...  
3  [1, لعمل, الكوكيز, :, تخلط, كل, المكونات, الجا...  
4  [1, لعمل, الصوص, :, في, وعاء, متوسط, العمق, عل...  


## 3. Parts of Speech (POS) <a id='part'></a>

In [283]:
nlp = stanza.Pipeline(lang='ar', processors='tokenize,pos')
def pos_tag_arabic(tokens):
    doc = nlp(' '.join(tokens))
    return [(word.text, word.pos) for sent in doc.sentences for word in sent.words]
df['recettes_pos_tags'] = df['recettes_tokenized'].apply(pos_tag_arabic)
print(df[['recettes', 'recettes_pos_tags']].head())

2024-04-08 04:43:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-08 04:43:40 INFO: Downloaded file to C:\Users\pc hp\stanza_resources\resources.json
2024-04-08 04:43:40 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 04:43:41 INFO: Loading these models for language: ar (Arabic):
| Processor | Package     |
---------------------------
| tokenize  | padt        |
| mwt       | padt        |
| pos       | padt_charlm |

2024-04-08 04:43:41 INFO: Using device: cpu
2024-04-08 04:43:41 INFO: Loading: tokenize
2024-04-08 04:43:41 INFO: Loading: mwt
2024-04-08 04:43:41 INFO: Loading: pos
2024-04-08 04:43:42 INFO: Done loading processors!


                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                   recettes_pos_tags  
0  [(1, NUM), (ي, PRON), (ُ, X), (خلط, NOUN), (‭,...  
1  [(1, NUM), (يخفق, VERB), (السكر, NOUN), (البود...  
2  [(1, NUM), (في, ADP), (وعاء, NOUN), (على, ADP)...  
3  [(1, NUM), (ل, ADP), (عمل, NOUN), (الكوكيز, NO...  
4  [(1, NUM), (ل, ADP), (عمل, NOUN), (الصوص, NOUN...  


In [284]:
print(df[['recettes', 'recettes_pos_tags']].head(50))

                                             recettes  \
0   \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1   \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2   \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3   \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4   \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   
5   \n\n\n1\n\nيخلط الطحين والقرفة والملح والسكر ف...   
6   \n\n\n1\n\nفي محضرة الطعام، يضرب دقيق اللوز مع...   
7   \n\n\n1\n\nتخلط كل المقادير معاً.\n\n\n\n\n\n2...   
8   \n\n\n1\n\nيغلى التمر والماء والسكر معاً، ثم ت...   
9   \n\n\n1\n\nفي وعاء خلاط كهربائي يوضع الزبد وال...   
10  \n\n\n1\n\nيسخن الفرن إلى درجة حرارة 350 فهرنه...   
11  \n\n\n1\n\nينخل الدقيق مع الملح والفلفل والزنج...   
12  \n\n\n1\n\nيصب الحليب في قدر ويوضع على النار إ...   
13  \n\n\n1\n\nتُخلط كل المكونات معاً في وعاء.\n\n...   
14  \n\n\n1\n\nتخفق الزبدة، وسكر الكاسترد والسكر ا...   
15  \n\n\n1\n\nيجمع المايونيز والجبنة والريحان وال...   
16  \n\n\n1\n\nتسخن ملعقة كبيرة

## 4. Named Entity Recognition(NER) <a id='ner'></a>

In [285]:
import spacy
nlp = spacy.load("xx_ent_wiki_sm")
def apply_ner(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]
df['recettes_ner'] = df['recettes'].apply(apply_ner)
print(df[['recettes', 'recettes_ner']].head())

                                            recettes  \
0  \n\n\n1\n\nيُخلط‭ ‬نوعي‭ ‬الدقيق‭ ‬والبيكنج‭ ‬...   
1  \n\n\n1\n\nيخفق السكر البودرة مع السكر الأبيض ...   
2  \n\n\n1\n\nفي وعاء على النار يخلط السكر مع الش...   
3  \n\n\n1\n\nلعمل الكوكيز : تخلط كل المكونات الج...   
4  \n\n\n1\n\nلعمل الصوص : في وعاء متوسط العمق عل...   

                                        recettes_ner  
0                                    [(يُضاف‭, PER)]  
1  [(بواسطة, PER), (متوسطة الحجم, PER), (كل عجينة...  
2  [(على درجة, PER), (متوسطة حتى يبدأ, MISC), (بو...  
3                  [(حتى, ORG), (بواسطة ملعقة, PER)]  
4               [(متوسط العمق على, PER), (حتى, ORG)]  


## 5. Lab Learning Recap <a id='recap'></a>

<p><strong>Through these labs, utilizing Beautiful Soup, I have gained a comprehensive understanding of web scraping techniques. I learned how to extract data from the Fatafeat cooking website, a specific domain, and store it in a NoSQL database such as MongoDB. Additionally, I established a Natural Language Processing (NLP) pipeline, which involved text cleaning, tokenization, removal of stop words, discretization, and normalization. Furthermore, I explored stemming and lemmatization techniques, comparing their effectiveness. I also delved into Parts of Speech (POS) techniques, including both rule-based and machine learning approaches, to analyze text data. Finally, I applied Named Entity Recognition (NER) methods to identify and classify entities in text data.</strong></p>